# Wikipedia Artist Detail Download

In [1]:
import os
import re
from urllib.parse import urljoin
import datetime as dt

import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import numpy as np
import scipy
import scipy.sparse

import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans

from IPython.display import display, HTML, clear_output

In [2]:
%run ./url_cache.ipynb
%run ./wikipedia_access.ipynb
%run ./wikipedia_album_detail_lib.ipynb

## Configure DB and Load Album Extract

In [3]:
db = sqlite3.Connection("var/data/music.db")

In [4]:
url_cache = URLContentCacheSqlite(db)

wayl_db = WikipediaAlbumYearListsDB(db)
albums = wayl_db.get_all()

### Get Artist Links

In [5]:
def get_artist_link(links):
    if links is not None and 'Artist' in links:
        return links['Artist'][0]['href']
    
albums['ArtistLink'] = albums.Links.apply(get_artist_link)

In [6]:
artists = albums[albums.ArtistLink.notnull()].drop_duplicates('Artist')[['Artist', 'ArtistLink']].copy().sort_values(by='Artist')

In [7]:
# See what was downloaded already
downloaded_urls = pd.read_sql(f"""
    select url
    from url_cache
""", db)

downloaded_urls = set(downloaded_urls.url)

artists['PrevDownloaded'] = artists.ArtistLink.isin(downloaded_urls)
print(f"Previously downloaded: {artists['PrevDownloaded'].sum()}")

d_count = artists['PrevDownloaded'].sum()

for _, artist_row in artists[~artists.PrevDownloaded].iterrows():
    print(f"[{d_count + 1}] Getting: {artist_row.Artist}")
    url_cache.get(artist_row.ArtistLink)
    d_count += 1
    if (d_count % 25) == 0:
        clear_output()

[11126] Getting: t.A.T.u.
[11127] Getting: tobyMac
[11128] Getting: tyDi
[11129] Getting: van Canto
[11130] Getting: will.i.am
[11131] Getting: yourcodenameis:milo
[11132] Getting: Álex Anwandter
[11133] Getting: Ásgeir
[11134] Getting: Ævangelist
[11135] Getting: Éric Lapointe
[11136] Getting: Ólafur Arnalds
[11137] Getting: Øystein Sevåg
[11138] Getting: Ünloco
[11139] Getting: …And You Will Know Us by the Trail of Dead
[11140] Getting: ≠Me


## Check An Arist Page

In [11]:
artist = artists[artists.Artist.str.contains("Run the Jewels")]
artist

,Artist,ArtistLink,PrevDownloaded
id,,,
2389,Run the Jewels,https://en.wikipedia.org/wiki/Run_the_Jewels,False


In [12]:
a_html = url_cache.get(artist.iloc[0].ArtistLink)['content']
display(HTML(a_html))